In [1]:
import numpy as np
import pyvista as pv
import random
#import math
import vtk
from vtk import vtkAppendPolyData, vtkPolyDataReader, vtkPolyDataWriter, vtkPolyData, vtkUnstructuredGridReader
from vtk.util.numpy_support import vtk_to_numpy


In [2]:
def write_vtk(points, cell, filename):
    with open(filename, 'w') as f:
        # Write header
        f.write("# vtk DataFile Version 4.2\n")
        f.write("vtk output\n")
        f.write("ASCII\n")
        f.write("DATASET POLYDATA\n")
        
        # Write points
        num_points = len(points)
        f.write("POINTS {} float\n".format(num_points))
        for point in points:
            f.write("{} {} {}\n".format(point[0], point[1], point[2]))
        
        # Write cells
        num_cells = len(cell)
        total_num_points = np.sum(cell[:, 0])
        f.write("POLYGONS {} {}\n".format(num_cells, total_num_points + num_cells))
        for cell_indices in cell:
            num_indices = cell_indices[0]
            f.write("{} ".format(num_indices))
            for index in cell_indices[1:]:
                f.write("{} ".format(index))
            f.write("\n")

def visualize(nameOfFile):
    # Load the VTK file
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(nameOfFile)
    reader.Update()

    # Create a mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(reader.GetOutputPort())

    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    # Create a renderer and add the actor to it
    renderer = vtk.vtkRenderer()
    renderer.AddActor(actor)

    # Create a render window and set the renderer as its active renderer
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.AddRenderer(renderer)
    renderWindow.SetSize(600, 600)
    renderer.ResetCamera()

    # Create an interactor and start the visualization
    interactor = vtk.vtkRenderWindowInteractor()
    interactor.SetRenderWindow(renderWindow)
    interactor.Start()
    
def readfile(filename):
    reader = vtk.vtkGenericDataObjectReader()
    reader.SetFileName(filename)
    reader.Update()

    polydata = reader.GetOutput()

    points = polydata.GetPoints()
    array = points.GetData()
    numpy_nodes = vtk_to_numpy(array)
    #print(numpy_nodes)
    cells = polydata.GetPolys()
    nCells = cells.GetNumberOfCells()
    array = cells.GetData()
    # This holds true if all polys are of the same kind, e.g. triangles.
    assert(array.GetNumberOfValues()%nCells==0)
    nCols = array.GetNumberOfValues()//nCells
    numpy_cells = vtk_to_numpy(array)
    numpy_cells = numpy_cells.reshape((-1,nCols))
    return numpy_cells, numpy_nodes

In [23]:
def generate_tetrahedron(edge_size = 3, mod = 0, noise = 0, img = 0):
    #This function generate tetrahedron
    #the output are the nodes, cells and Pyvista mesh
    #for this function you need numpy and pyvista modules
    vertices = np.array([[0, 0, 0], [edge_size, 0, 0], [0, edge_size, 0], [0, 0, edge_size]])

    # Define the indices of the faces of the tetrahedron
    faces = np.array([[0, 2, 1], [0, 1, 3], [0, 3, 2], [1, 2, 3]])

    # Calculate the center point of each face
    face_centers = np.array([vertices[face].mean(axis=0) for face in faces])
    

    # Combine the vertices and face centers to create the final set of points
    points = np.concatenate((vertices, face_centers) )
    
    
    # Manually mesh and cells
    mesh = np.array([[3, 0, 1, 4], [3, 0, 4, 2], [3, 2, 4, 1], #1
                     [3, 0, 5, 3], [3, 0, 5, 1], [3, 5, 3, 1], #2
                     [3, 0, 6, 2], [3, 0, 6, 3], [3, 6, 3, 2], #3
                     [3, 3, 7, 1], [3, 3, 7, 2], [3, 7, 2, 1], #4
                     ])
    
    #calculate original bigger faces normalvectors
    normals = []
    for f in faces:
        v1 = vertices[f[1]] - vertices[f[0]]
        v2 = vertices[f[2]] - vertices[f[0]]
        normal = np.cross(v1, v2)
        normals.append(normal)

    normals = np.array(normals)
    for v in range(len(normals)):
        n = np.linalg.norm(normals[v])
        normals[v] = normals[v] / n
    
    #if the mod input is a non-zero number, the face center points shift for the normal vector positive or negative direction
    if mod != 0:
        for p in range(len(face_centers)):
            r = random.uniform(-edge_size/3 +0.1, edge_size/3 -0.1)
            face_centers[p] = face_centers[p] + normals[p]*r
            
        new_points = np.concatenate((vertices, face_centers))
        print(points, new_points)
    else:
        #else they don't shift
        new_points = np.concatenate((vertices, face_centers))
    
    #if the noise input is non-zero number, each point of the mesh got noise, that means random values are added to each coordinate
    if noise != 0:
        new_points = new_points + np.random.random((len(new_points), 3)) / 9.21 * np.random.choice([-1, 1], size=(len(new_points), 3))
        
    #else there isn't noise
    else:
        new_points = np.concatenate((vertices, face_centers))
        
    #create generated mesh with pyvista module
    #add the array of points/nodes and array of cells
    genMesh = pv.PolyData(new_points, mesh)

    # Plot the mesh if img is non-zero
    if img != 0:
        plotter = pv.Plotter(notebook = 0)
        plotter.show_axes()
        plotter.add_mesh(genMesh, show_edges=True)
        plotter.show()
    
    
    
    return mesh, new_points, genMesh


In [26]:
#cells_10, nodes_10, Mesh_10 = generate_tetrahedron(edge_size = 10,mod= 1, noise = 1, img = 1)
cells_1, nodes_1, Mesh_1 = generate_tetrahedron(edge_size = 3,mod= 0, noise = 0, img = 1)
#shift = np.array([0,0,13])
#nodes_1 = nodes_1 + shift

From here this is the generated dataset for the classification..

In [54]:
#generate 80 regular tetrahedreron shapes
ALL_regular_cells = []
ALL_regular_nodes = []
ALL_regular_Meshes = []
regular_edge_size = np.random.uniform(low=3, high=10, size=80)
for i in range(len(regular_edge_size)):
    cells, nodes, Mesh = generate_tetrahedron(edge_size = regular_edge_size[i] ,mod= 0, noise = 0, img = 0)
    ALL_regular_cells.append(cells)
    ALL_regular_nodes.append(nodes)
    ALL_regular_Meshes.append(Mesh)
    
#generate 20 unregular tetrahedreron shapes
ALL_UNregular_cells = []
ALL_UNregular_nodes = []
ALL_UNregular_Meshes = []
UNregular_edge_size = np.random.uniform(low=3.1, high=10.1, size=20)
for i in range(len(UNregular_edge_size)):
    cells, nodes, Mesh = generate_tetrahedron(edge_size = UNregular_edge_size[i] ,mod= 0, noise = 1, img = 0)
    ALL_UNregular_cells.append(cells)
    ALL_UNregular_nodes.append(nodes)
    ALL_UNregular_Meshes.append(Mesh)

#the cells are same in every case
ALL_cells = ALL_regular_cells + ALL_UNregular_cells
#the nodes are NOT same
ALL_nodes = ALL_regular_nodes + ALL_UNregular_nodes
#
ALL_Meshes = ALL_regular_Meshes + ALL_UNregular_Meshes



In [ ]:
plotter = pv.Plotter(notebook = 0)
plotter.show_axes()
plotter.add_mesh(ALL_Meshes[5], show_edges=True)
plotter.show()

In [9]:
#to save the shape in vtk file 

write_vtk(nodes_10, cells_10, "thetraeder_10.vtk")
write_vtk(nodes_1, cells_1, "thetraeder_1.vtk")

#visualize("thetraeder.vtk")

In [10]:
reader = vtkPolyDataReader()#vtkUnstructuredGridReader()vtkPolyDataReader()
append = vtkAppendPolyData()

filenames = ['thetraeder_10.vtk', 'thetraeder_1.vtk']
for file in filenames:
    reader.SetFileName(file)
    reader.Update()
    polydata = vtkPolyData()
    polydata.ShallowCopy(reader.GetOutput())
    append.AddInputData(polydata)

append.Update()    

writer = vtkPolyDataWriter()
writer.SetFileName('output.vtk')
writer.SetInputData(append.GetOutput())
writer.Write()

1

In [7]:
#visualize("output.vtk")

In [11]:
c, n = readfile("output.vtk")
genMesh = pv.PolyData(n, c)

# Plot the mesh if img is non-zero
plotter = pv.Plotter(notebook = 0)
plotter.show_axes()
plotter.add_mesh(genMesh, show_edges=True)
plotter.show()

In [32]:
len(cells_1)

12

In [17]:
np.random.random(1)

array([0.21756379])